In [54]:
import sklearn
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import urllib
import spacy
from tqdm import tqdm
import en_core_web_sm
import re
import itertools
from IPython.display import display, HTML

In [55]:
# Init
tqdm.pandas()
pd.set_option('display.max_colwidth', -1)

/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [159]:
ARG1 = "$ARG1"
ARG2 = "$ARG2"
FINAL_DF_COLUMNS = ['sample', 'extr_sample', 'pattern', 'weak_label', 'gold_label']

Upload the CONLL Data

In [57]:
conll_file = "dev.conll"
with open(conll_file, "r+") as input_file:
    train_data = input_file.read()

The first step is to define the relations we want to be extracted in our dataset:

In [58]:
def process_data(path_to_data):
    samples, relations = [], []
    with open(path_to_data, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith("# id="):    # Instance starts
                sample = ""
                label = line.split(" ")[3][5:]
            elif line == "":  # Instance ends
                samples.append(sample)
                relations.append(label)
            elif line.startswith("#"):  # comment
                continue
            else:
                parts = line.split("\t")
                token = parts[1]
                if token == "-LRB-":
                    token = "("
                elif token == "-RRB-":
                    token = ")"
                sample += " " + token
    return samples, relations

samples, labels = process_data(conll_file)

Let's inspect all relation labels that we have in our dataset

In [59]:
print(set(labels))

{'org:country_of_headquarters', 'per:origin', 'per:city_of_birth', 'org:member_of', 'per:children', 'per:stateorprovinces_of_residence', 'per:alternate_names', 'per:spouse', 'per:parents', 'per:country_of_death', 'org:founded', 'per:age', 'org:top_members/employees', 'per:siblings', 'no_relation', 'per:cities_of_residence', 'per:country_of_birth', 'org:shareholders', 'per:cause_of_death', 'per:stateorprovince_of_birth', 'per:other_family', 'per:countries_of_residence', 'per:city_of_death', 'org:dissolved', 'org:website', 'org:parents', 'per:date_of_death', 'per:charges', 'org:founded_by', 'org:political/religious_affiliation', 'per:date_of_birth', 'per:stateorprovince_of_death', 'org:number_of_employees/members', 'org:alternate_names', 'org:members', 'per:employee_of', 'per:religion', 'org:stateorprovince_of_headquarters', 'org:city_of_headquarters', 'per:schools_attended', 'org:subsidiaries', 'per:title'}


In [60]:
samples = pd.DataFrame.from_dict({"samples": samples, "labels": labels})
samples.head()

,samples,labels
0,"At the same time , Chief Financial Officer Douglas Flint will become chairman , succeeding Stephen Green who is leaving to take a government job .",per:title
1,U.S. District Court Judge Jeffrey White in mid-February issued an injunction against Wikileaks after the Zurich-based Bank Julius Baer accused the site of posting sensitive account information stolen by a disgruntled former employee .,no_relation
2,"PARIS 2009-07-07 11:07:32 UTC French media earlier reported that Montcourt , ranked 119 , was found dead by his girlfriend in the stairwell of his Paris apartment .",per:city_of_death
3,"The current holdings of Blackstone-operated funds include Universal Orlando , Cadbury Schweppes , Freedom Communications , Nielsen Co. , Orangina and Vanguard Health Systems .",no_relation
4,The ICDC was formed after the Nepali government and the guerrillas reached in an understanding during summit talks held on July 16 at Prime Minister Girija Prashad Koirala 's residence at Baluwatar in downtown Kathmandu city .,no_relation


To make the calculation quicklier, let's choose only the samples that contain a relation (that is, labelled not with "no_relation" label). Moreover, let's reduce the number of relation we want to find and choose 5 relations from the TACRED relation labels.

In [61]:
selected_labels = {'org:alternate_names', 'per:date_of_birth', "org:top_members/employees"} '#not used
# relevant_samples = samples[samples['labels'].isin(selected_labels)]
# samples.groupby('labels').count()

In [62]:
# todo: simplify
relevant_samples = samples[samples["labels"]!='no_relation'].sample(n=400)
relevant_samples.groupby('labels').count()

,samples
labels,
org:alternate_names,19
org:city_of_headquarters,8
org:country_of_headquarters,10
org:dissolved,1
org:founded,2
org:founded_by,5
org:member_of,2
org:members,9
org:number_of_employees/members,2


In [63]:
relevant_samples.head()

,samples,labels
14713,"ALBA -- the Bolivarian Alternative for the Americas -- was founded by Venezuelan President Hugo Chavez and Cuban leader Fidel Castro in 2004 and also includes Bolivia , Nicaragua and the Caribbean island of Dominica .",org:members
5313,"His conversion to Islam came after he attended the Islamic Centre of Orange County , California , where he is believed to have come under the influence of two foreign-born Islamist radicals .",per:religion
12148,The United Nations appointed two foreign members of Afghanistan 's Election Complaints Commission ( ECC ) here on Saturday .,org:alternate_names
17841,"Marks & Spencer has said the move will give Rose time to groom a new chief executive to replace him by 2011 , but some investors say the move concentrates too much power in one person .",per:title
15602,"She was the poet laureate of Maryland from 1979 to 1985 , a finalist for the Pulitzer Prize in 1988 and a visiting professor at Columbia and Duke universities .",per:title


In order to turn the data into distantly supervised one, let's write down for each relation a couple of simple patterns that could help us to find the relations


In [172]:
patterns = pd.DataFrame.from_dict({"org:alternate_names": 
                                   [["$ARG1 ( $ARG2 ", 
                                     "$ARG1 formerly known as $ARG2", 
                                     "$ARG1 aka $ARG2", 
                                     "$ARG1 ( also known as $ARG2 )"]], 
                                   "per:date_of_birth": 
                                   [["$ARG1 ( born $ARG2 )", 
                                    "$ARG1 ( born $ARG2 in", 
                                    "$ARG1 ( $ARG2 -", 
                                    "$ARG1 was born in $ARG2"]],
                                   "org:top_members/employees":
                                   [["$ARG1 , executive director of $ARG2", 
                                     "$ARG1 , head of $ARG2", 
                                     "$ARG1 , who heads $ARG2",
                                     "$ARG1 , chief executive of $ARG2"]]}, orient='index', columns = ["Raw patterns"])

In [173]:
def escape_dollar(string):
    return re.sub("\\$", "\\\\$", str(string))

In [174]:
patterns.apply(lambda x: [escape_dollar(pattern) for pattern in x]).head()

,Raw patterns
org:alternate_names,"['\$ARG1 ( \$ARG2 ', '\$ARG1 formerly known as \$ARG2', '\$ARG1 aka \$ARG2', '\$ARG1 ( also known as \$ARG2 )']"
per:date_of_birth,"['\$ARG1 ( born \$ARG2 )', '\$ARG1 ( born \$ARG2 in', '\$ARG1 ( \$ARG2 -', '\$ARG1 was born in \$ARG2']"
org:top_members/employees,"['\$ARG1 , executive director of \$ARG2', '\$ARG1 , head of \$ARG2', '\$ARG1 , who heads \$ARG2', '\$ARG1 , chief executive of \$ARG2']"


Since we want to make a simple regex search, convert patterns into regexes

In [175]:
def preprocess_patterns(patterns):
    regex_patterns = [re.sub("\\\\\\$ARG", "(A )?(a )?(The )?(the )?\\$ARG", re.escape(pattern)) for pattern in patterns]
    return regex_patterns

patterns["Regex patterns"] = patterns["Raw patterns"].apply(preprocess_patterns)
patterns.apply(lambda x: [escape_dollar(pattern) for pattern in x]).head()

,Raw patterns,Regex patterns
org:alternate_names,"['\$ARG1 ( \$ARG2 ', '\$ARG1 formerly known as \$ARG2', '\$ARG1 aka \$ARG2', '\$ARG1 ( also known as \$ARG2 )']","['(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ ', '(A )?(a )?(The )?(the )?\\\$ARG1\\ formerly\\ known\\ as\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ aka\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ also\\ known\\ as\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ \\)']"
per:date_of_birth,"['\$ARG1 ( born \$ARG2 )', '\$ARG1 ( born \$ARG2 in', '\$ARG1 ( \$ARG2 -', '\$ARG1 was born in \$ARG2']","['(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ born\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ \\)', '(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ born\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ in', '(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ \\-', '(A )?(a )?(The )?(the )?\\\$ARG1\\ was\\ born\\ in\\ (A )?(a )?(The )?(the )?\\\$ARG2']"
org:top_members/employees,"['\$ARG1 , executive director of \$ARG2', '\$ARG1 , head of \$ARG2', '\$ARG1 , who heads \$ARG2', '\$ARG1 , chief executive of \$ARG2']","['(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ executive\\ director\\ of\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ head\\ of\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ who\\ heads\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ chief\\ executive\\ of\\ (A )?(a )?(The )?(the )?\\\$ARG2']"


Now we can simply search patterns in the sentences. Because of the speed reasons, let's take only samples where we will definetly find smth :)

In [69]:
nlp = spacy.load("en_core_web_sm")
relevant_samples["Spacy_info"] = relevant_samples["samples"].apply(lambda x: nlp(x).to_json())

In [70]:
relevant_samples.apply(lambda x: [escape_dollar(pattern) for pattern in x]).head()

samples  \
14713   ALBA -- the Bolivarian Alternative for the Americas -- was founded by Venezuelan President Hugo Chavez and Cuban leader Fidel Castro in 2004 and also includes Bolivia , Nicaragua and the Caribbean island of Dominica .   
5313    His conversion to Islam came after he attended the Islamic Centre of Orange County , California , where he is believed to have come under the influence of two foreign-born Islamist radicals .                             
12148   The United Nations appointed two foreign members of Afghanistan 's Election Complaints Commission ( ECC ) here on Saturday .                                                                                                
17841   Marks & Spencer has said the move will give Rose time to groom a new chief executive to replace him by 2011 , but some investors say the move concentrates too much power in one person .                                   
15602   She was the poet laureate of Maryland from 1979 to 1985 , a finalist for the Pulitzer Prize in 1988 and a visiting professor at Columbia and Duke universities .                                                            

                    labels  \
14713  org:members           
5313   per:religion          
12148  org:alternate_names   
17841  per:title             
15602  per:title             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

After that, take the arguments pairwise in each sentence and try if there is a pattern match

In [71]:
def get_extracted_sample(sample):
    return [(ARG1 + sample["text"][ent1["end"]:ent2["start"]] + ARG2) if ent1["end"] < ent2["end"] 
            else (ARG2 + sample["text"][ent2["end"]:ent1["start"]] + ARG1)
            for ent1, ent2 in itertools.permutations(sample["ents"],2)]

relevant_samples["candidates"] = relevant_samples["Spacy_info"].apply(lambda x: get_extracted_sample(x))

In [72]:
relevant_samples.apply(lambda x: [escape_dollar(pattern) for pattern in x]).head()

samples  \
14713   ALBA -- the Bolivarian Alternative for the Americas -- was founded by Venezuelan President Hugo Chavez and Cuban leader Fidel Castro in 2004 and also includes Bolivia , Nicaragua and the Caribbean island of Dominica .   
5313    His conversion to Islam came after he attended the Islamic Centre of Orange County , California , where he is believed to have come under the influence of two foreign-born Islamist radicals .                             
12148   The United Nations appointed two foreign members of Afghanistan 's Election Complaints Commission ( ECC ) here on Saturday .                                                                                                
17841   Marks & Spencer has said the move will give Rose time to groom a new chief executive to replace him by 2011 , but some investors say the move concentrates too much power in one person .                                   
15602   She was the poet laureate of Maryland from 1979 to 1985 , a finalist for the Pulitzer Prize in 1988 and a visiting professor at Columbia and Duke universities .                                                            

                    labels  \
14713  org:members           
5313   per:religion          
12148  org:alternate_names   
17841  per:title             
15602  per:title             

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [160]:
def pattern_search(sample_sent, patterns, row):
    for relation, rel_patterns in patterns.iterrows():
        matches = [[row["samples"], sample_sent, p, relation, row["labels"]] 
                   for p in rel_patterns["Regex patterns"] 
                   if re.match(p, sample_sent) is not None]
        if len(matches) > 0:
            return pd.DataFrame(matches, columns = FINAL_DF_COLUMNS)

In [161]:
all_matches = pd.DataFrame(columns = FINAL_DF_COLUMNS)
for _, row in relevant_samples.iterrows():
    for cand_samples in row["candidates"]:
        df_found = pattern_search(cand_samples, patterns, row)
        if isinstance(df_found, pd.DataFrame) and not df_found.empty:
            all_matches = pd.concat([all_matches, df_found])

In [162]:
all_matches.apply(lambda x: [escape_dollar(pattern) for pattern in x]).head()

,sample,extr_sample,pattern,weak_label,gold_label
0,"Anatoly Isaikin , head of Russia 's state-run arms exporter Rosoboronexport , told a news conference Russia would consider exports only after meeting the requirements of its own armed forces .","\$ARG1 , head of \$ARG2","(A )?(a )?(The )?(the )?\\$ARG1\ ,\ head\ of\ (A )?(a )?(The )?(the )?\\$ARG2",org:top_members/employees,org:top_members/employees
0,"But Sergey Chemezov , head of the Russian arms export company Rosoboronexport denied the report .","\$ARG1 , head of the \$ARG2","(A )?(a )?(The )?(the )?\\$ARG1\ ,\ head\ of\ (A )?(a )?(The )?(the )?\\$ARG2",org:top_members/employees,org:top_members/employees
0,"Ronald James Padavona was born in Portsmouth , NH , and grew up in Cortland , NY .",\$ARG1 was born in \$ARG2,(A )?(a )?(The )?(the )?\\$ARG1\ was\ born\ in\ (A )?(a )?(The )?(the )?\\$ARG2,per:date_of_birth,per:cities_of_residence


But here we can observe some misclassified sentences: for example, a sentence 

"A professor emeritus at Yale University , Mandelbrot was born in Poland but as a child moved with his family to France where he was educated" 

was assigned with a label "per:date_of_birth" (presumably by a pattern "ARG1 was born in ARG2"), what is definitely wrong. In order to avoid such mistake, let's add additional constraints on the argument types.

In [176]:
relation_to_types = {"org:alternate_names": ['PERSON', 'PERSON'], 
                     "per:date_of_birth": ['PERSON', 'DATE'],
                     "org:top_members/employees": ['PERSON', 'ORG']}

So, when we look for these patterns in samples, we should take into account the entity types referred to corresponding relation

In [177]:
patterns["types"] = pd.Series(relation_to_types)

In [178]:
patterns.apply(lambda x: [escape_dollar(pattern) for pattern in x]).head()

,Raw patterns,Regex patterns,types
org:alternate_names,"['\$ARG1 ( \$ARG2 ', '\$ARG1 formerly known as \$ARG2', '\$ARG1 aka \$ARG2', '\$ARG1 ( also known as \$ARG2 )']","['(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ ', '(A )?(a )?(The )?(the )?\\\$ARG1\\ formerly\\ known\\ as\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ aka\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ also\\ known\\ as\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ \\)']","['PERSON', 'PERSON']"
per:date_of_birth,"['\$ARG1 ( born \$ARG2 )', '\$ARG1 ( born \$ARG2 in', '\$ARG1 ( \$ARG2 -', '\$ARG1 was born in \$ARG2']","['(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ born\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ \\)', '(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ born\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ in', '(A )?(a )?(The )?(the )?\\\$ARG1\\ \\(\\ (A )?(a )?(The )?(the )?\\\$ARG2\\ \\-', '(A )?(a )?(The )?(the )?\\\$ARG1\\ was\\ born\\ in\\ (A )?(a )?(The )?(the )?\\\$ARG2']","['PERSON', 'DATE']"
org:top_members/employees,"['\$ARG1 , executive director of \$ARG2', '\$ARG1 , head of \$ARG2', '\$ARG1 , who heads \$ARG2', '\$ARG1 , chief executive of \$ARG2']","['(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ executive\\ director\\ of\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ head\\ of\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ who\\ heads\\ (A )?(a )?(The )?(the )?\\\$ARG2', '(A )?(a )?(The )?(the )?\\\$ARG1\\ ,\\ chief\\ executive\\ of\\ (A )?(a )?(The )?(the )?\\\$ARG2']","['PERSON', 'ORG']"


In [345]:
final_samples_with_entity_types = pd.DataFrame(columns=['Sentence', 'Pattern', 'Label'])

for sample in parsed_samples:
    sentence = sample["text"]
    for ent1, ent2 in itertools.permutations(sample["ents"],2):
        entity_types = [ent1["label"], ent2["label"]]
        if entity_types in relation_to_types.values():
            curr_relation = [rel for rel,types in relation_to_types.items() if types==entity_types][0]
            extr_sentences = [get_extracted_sample(ARG1, ARG2, ent1, ent2, sentence) if ent1["end"] < ent2["end"] 
                              else get_extracted_sample(ARG2, ARG1, ent2, ent1, sentence)
                              for ent1, ent2 in itertools.permutations(sample["ents"],2)]
            for sent in extr_sentences:
                for pattern in relation_patterns_regex[curr_relation]:
                    if re.match(pattern, sent) is not None:
                        match = [sentence, re.escape(regex_pattern_to_regex[pattern]), curr_relation] 
                        a_series = pd.Series(match, index = final_samples_with_entity_types.columns)
                        final_samples_with_entity_types = final_samples_with_entity_types.append(a_series, ignore_index=True)

In [346]:
pd.set_option('display.max_rows', len(final_samples_with_entity_types))
print(display(final_samples_with_entity_types))

,Sentence,Pattern,Label
0,"Gwathmey was born in 1938 , the only child of painter Robert Gwathmey and his wife , Rosalie , a photographer .",\$ARG1\ was\ born\ in\ \$ARG2,per:date_of_birth
1,"Gwathmey was born in 1938 , the only child of painter Robert Gwathmey and his wife , Rosalie , a photographer .",\$ARG1\ was\ born\ in\ \$ARG2,per:date_of_birth
2,"Gwathmey was born in 1938 , the only child of painter Robert Gwathmey and his wife , Rosalie , a photographer .",\$ARG1\ was\ born\ in\ \$ARG2,per:date_of_birth
3,"Khan was born April 14 , 1922 , in Shivpur , East Bengal ( now Bangladesh ) .",\$ARG1\ was\ born\ \$ARG2,per:date_of_birth
4,"`` The model would generate tax revenues while respecting the privacy of bank clients and it would represent an efficient alternative to a system of automatic information exchange , '' said Urs Roth , chief executive of the Swiss Bankers Association which has almost 360 members from the financial industry .","\$ARG1\ ,\ chief\ executive\ of\ \$ARG2",org:top_members/employees


None


Let's try more sophisticated way: not just look for patterns in samples, but to get the entities first and look at the more precise relation between them. We are going to do it with SpaCy package. 